In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash  # Use Dash(__name__) if deploying standalone

# Import dashboard components
import dash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import base64
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the enhanced local CRUD module
from animalshelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "Jnasr"
password = "Password123"
shelter = AnimalShelter(username, password)

# Load data with error handling
try:
    data = list(shelter.read({}))
    df = pd.DataFrame.from_records(data)
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    print("Data loaded successfully.")
except Exception as e:
    print(f"Failed to load data: {e}")
    df = pd.DataFrame()

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load logo image
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height=220, width=220))
    ]),
    html.Center(html.B(html.H1('Jason Nasr - CS-340 Dashboard'))),
    html.Hr(),

    html.Div([
        dcc.Dropdown(
            id='dropdown_filter',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MountainWild'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'DisasterInd'},
                {'label': 'Reset', 'value': 'Reset'},
            ],
            value='Reset'
        ),
    ]),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        style_table={'height': '100%', 'overflowY': 'auto'},
        page_size=10,
        sort_mode="multi",
        sort_action="native",
        row_selectable="single",
        selected_rows=[0],
        filter_action="native",
        page_action="native",
        page_current=0,
    ),

    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([
    Output('datatable-id', 'data'),
    Output('datatable-id', 'columns')
],
    Input('dropdown_filter', 'value')
)
def update_dashboard(value):
    if value == 'Water':
        criteria = {
            "breed": {"$in": ["Chesapeake Bay Retriever", "Labrador Retriever Mix", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }
    elif value == 'MountainWild':
        criteria = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }
    elif value == 'DisasterInd':
        criteria = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        }
    else:
        criteria = {}

    data = list(shelter.read(criteria))
    df = pd.DataFrame.from_records(data)
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    return df.to_dict('records'), columns

@app.callback(
    Output('graph-id', 'children'),
    Input('datatable-id', 'derived_virtual_data')
)
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [dcc.Graph(figure=px.pie(df, names='breed', title='Animals by Breed'))]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns')
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns or []]

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    if not index:
        row = 0
    else:
        row = index[0]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 9]),
                dl.Popup([html.H1("Name"), html.P(dff.iloc[row, 9])])
            ])
        ])
    ]

# Run the app
app.run_server(debug=True)

ModuleNotFoundError: No module named 'jupyter_dash'